## Import Libraries and Data

In [1]:
import spacy
import pandas as pd
import numpy as np
import re
from sklearn.decomposition import PCA
#For counting of tokens
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(stop_words = 'english', max_df = 2)
#Load english corpus
en_nlp = spacy.load('en')
#Add additional stopwords
more_stopwords = ['ziisha', 'ziidisha', 'zidsisha',
       'zidishq', 'zidishia', 'zidishi', 'zidisher', 'zidishans',
       'zidisaha', 'zidisa', 'zidihsa', 'zididsha', 'zididisha', 'zididha',
       'zidiasha', 'ziddisha', 'zdisha', 'loans', 'loan', 'thank', 'you', 
        'business', 'funding', 'fund']
for word in more_stopwords:
    en_nlp.vocab[word].is_stop = True
#Load data from Stefan
dat = pd.read_csv('Regression_Dataset_20161221(na).csv')
#so you can see more columns
pd.options.display.max_columns = 500
pd.options.display.max_rows = 200

%matplotlib inline

## Define Helper Functions

In [2]:
def get_top_tokens(pandas_series, n=200):
    """
    gives you a count of most common tokens in text field
    """
    
    vect.fit(pandas_series.dropna())
    counts = list(vect.vocabulary_.items())
    df = pd.DataFrame(counts, columns=['token', 'count']).set_index('token').sort_values('count', ascending=False)
    return df.head(n)


def clean_text(text_string):
    """ 
    remove punctuation, convert to lowercase and remove stop words
    """
    doc =  en_nlp(text_string)
    return(' '.join([token.text.lower() for token in doc if token.is_stop == False and token.is_punct ==False]))


def vectorize_text(text_string):
    """
    return GloVe vectors for words, document vector is the average of the word vectors
    reference: http://nlp.stanford.edu/projects/glove/
    
    spacy library tokenizes, cleans, etc. so you don't have to do this yourself. 
    """
    if type(text_string) != str:
        return np.zeros(300)  # return array of 300 zeros as built-in GloVe embedding has 300 dimensions
    
    doc =  en_nlp(text_string)
    return np.round(doc.vector, 3) #reduce size of datatset by reducing digits

In [3]:
def attach_vectorized_column(df, column_name_list):
    """
    append vectorized GloVe columns to dataframe
    
    warning: will add 300 columns for each text field
    """
    dataframes = []
    dataframes.append(df)
    
    for column_name in column_name_list:
        print('parsing {}'.format(column_name))
        
        col_names = [column_name+'_txtvec_'+str(x) for x in range(1, 301)]
        
        vecdf = pd.DataFrame.from_records((df[column_name].
        apply(vectorize_text)),
        columns = col_names)
        
        dataframes.append(vecdf)
    
    return pd.concat(dataframes, axis = 1)

In [4]:
## Save for later
#pca = PCA(n_components=25)
#pca.fit(df)
#for i,v in enumerate(np.cumsum(pca.explained_variance_ratio_)):
#    print(i,v)

# Preview Dataset

In [5]:
dat.head()

,id,borrower_id,usd_amount,len_proposal,len_about_me,len_about_business,len_address,missing_natl_id,missing_referred_by,application_time,default_flag,fraud_flag,nonfraud_default,friends_count,country_id,name,category_id,invited_flag,made_pmts,missed_pmts,sift_science_score,english_flag,business_years,usd_installment_amount,reserve_fee_pct,sift_labeled_bad,prior_loans,country_internet_users,country_life_expectancy,country_literacy,country_gdppc,about_me_field,about_business_field,proposal_field
0,4863,15031,50.0,422,573.0,835.0,16,0,1,NaN,0,0,0,41.0,32,Kenya,NaN,0,13,0,NaN,1,NaN,NaN,0.0,1,0,46,62,78,1429,My\r\n name is Julius Moturi aged 44 and i run...,My\r\n business revolves around sourcing and s...,The loan will be used in purchasing of drugs a...
1,5687,15031,150.0,748,573.0,835.0,16,0,1,NaN,1,0,1,41.0,32,Kenya,NaN,0,10,8,NaN,1,NaN,NaN,0.0,1,1,46,62,78,1429,My\r\n name is Julius Moturi aged 44 and i run...,My\r\n business revolves around sourcing and s...,The loan will be used to purchase more drugs f...
2,4895,15057,250.0,504,1374.0,627.0,15,0,1,NaN,0,0,0,NaN,32,Kenya,NaN,1,41,0,NaN,1,NaN,NaN,0.0,0,0,46,62,78,1429,I grew up in Nyamira Count in the Republic of ...,I will spend the loan money on purchasing drug...,"First and foremost, i would want to take this ..."
3,10181,15057,244.0,627,1374.0,627.0,15,0,1,NaN,0,0,0,NaN,32,Kenya,9.0,1,35,0,NaN,1,NaN,NaN,0.0,0,1,46,62,78,1429,I grew up in Nyamira Count in the Republic of ...,I will spend the loan money on purchasing drug...,I will spend the loan money on purchasing drug...
4,20720,15057,220.0,627,1374.0,627.0,15,0,1,NaN,0,0,0,NaN,32,Kenya,9.0,1,32,0,0.7,1,NaN,31.08,0.0,0,2,46,62,78,1429,I grew up in Nyamira Count in the Republic of ...,I will spend the loan money on purchasing drug...,I will spend the loan money on purchasing drug...


#### See top tokens in proposal field

In [6]:
get_top_tokens(dat.proposal_field, 20)

,count
token,
ön,18308
être,18307
été,18306
énorme,18305
élevage,18304
électricité,18303
écouler,18302
zte,18301
zorgho,18300


# Vectorize Text Fields

Using this technique called Glove to vectorize text:  http://nlp.stanford.edu/projects/glove/

Using this library:  https://spacy.io/  , where GloVe has been pretrained on the Common Crawl corpus

#### Warning:  computing the vectors can take 15-30 minutes, and is fairly memory hungry; if you are playing with this maybe just skip this part and load from csv

In [7]:
df = attach_vectorized_column(dat, ['about_me_field', 'about_business_field', 'proposal_field'])

parsing about_me_field
parsing about_business_field
parsing proposal_field


**Preview dataset with vectorized columns**

In [9]:
df.head()

,id,borrower_id,usd_amount,len_proposal,len_about_me,len_about_business,len_address,missing_natl_id,missing_referred_by,application_time,default_flag,fraud_flag,nonfraud_default,friends_count,country_id,name,category_id,invited_flag,made_pmts,missed_pmts,sift_science_score,english_flag,business_years,usd_installment_amount,reserve_fee_pct,sift_labeled_bad,prior_loans,country_internet_users,country_life_expectancy,country_literacy,country_gdppc,about_me_field,about_business_field,proposal_field,about_me_field_txtvec_1,about_me_field_txtvec_2,about_me_field_txtvec_3,about_me_field_txtvec_4,about_me_field_txtvec_5,about_me_field_txtvec_6,about_me_field_txtvec_7,about_me_field_txtvec_8,about_me_field_txtvec_9,about_me_field_txtvec_10,about_me_field_txtvec_11,about_me_field_txtvec_12,about_me_field_txtvec_13,about_me_field_txtvec_14,about_me_field_txtvec_15,about_me_field_txtvec_16,about_me_field_txtvec_17,about_me_field_txtvec_18,about_me_field_txtvec_19,about_me_field_txtvec_20,about_me_field_txtvec_21,about_me_field_txtvec_22,about_me_field_txtvec_23,about_me_field_txtvec_24,about_me_field_txtvec_25,about_me_field_txtvec_26,about_me_field_txtvec_27,about_me_field_txtvec_28,about_me_field_txtvec_29,about_me_field_txtvec_30,about_me_field_txtvec_31,about_me_field_txtvec_32,about_me_field_txtvec_33,about_me_field_txtvec_34,about_me_field_txtvec_35,about_me_field_txtvec_36,about_me_field_txtvec_37,about_me_field_txtvec_38,about_me_field_txtvec_39,about_me_field_txtvec_40,about_me_field_txtvec_41,about_me_field_txtvec_42,about_me_field_txtvec_43,about_me_field_txtvec_44,about_me_field_txtvec_45,about_me_field_txtvec_46,about_me_field_txtvec_47,about_me_field_txtvec_48,about_me_field_txtvec_49,about_me_field_txtvec_50,about_me_field_txtvec_51,about_me_field_txtvec_52,about_me_field_txtvec_53,about_me_field_txtvec_54,about_me_field_txtvec_55,about_me_field_txtvec_56,about_me_field_txtvec_57,about_me_field_txtvec_58,about_me_field_txtvec_59,about_me_field_txtvec_60,about_me_field_txtvec_61,about_me_field_txtvec_62,about_me_field_txtvec_63,about_me_field_txtvec_64,about_me_field_txtvec_65,about_me_field_txtvec_66,about_me_field_txtvec_67,about_me_field_txtvec_68,about_me_field_txtvec_69,about_me_field_txtvec_70,about_me_field_txtvec_71,about_me_field_txtvec_72,about_me_field_txtvec_73,about_me_field_txtvec_74,about_me_field_txtvec_75,about_me_field_txtvec_76,about_me_field_txtvec_77,about_me_field_txtvec_78,about_me_field_txtvec_79,about_me_field_txtvec_80,about_me_field_txtvec_81,about_me_field_txtvec_82,about_me_field_txtvec_83,about_me_field_txtvec_84,about_me_field_txtvec_85,about_me_field_txtvec_86,about_me_field_txtvec_87,about_me_field_txtvec_88,about_me_field_txtvec_89,about_me_field_txtvec_90,about_me_field_txtvec_91,about_me_field_txtvec_92,about_me_field_txtvec_93,about_me_field_txtvec_94,about_me_field_txtvec_95,about_me_field_txtvec_96,about_me_field_txtvec_97,about_me_field_txtvec_98,about_me_field_txtvec_99,about_me_field_txtvec_100,about_me_field_txtvec_101,about_me_field_txtvec_102,about_me_field_txtvec_103,about_me_field_txtvec_104,about_me_field_txtvec_105,about_me_field_txtvec_106,about_me_field_txtvec_107,about_me_field_txtvec_108,about_me_field_txtvec_109,about_me_field_txtvec_110,about_me_field_txtvec_111,about_me_field_txtvec_112,about_me_field_txtvec_113,about_me_field_txtvec_114,about_me_field_txtvec_115,about_me_field_txtvec_116,about_me_field_txtvec_117,about_me_field_txtvec_118,about_me_field_txtvec_119,about_me_field_txtvec_120,about_me_field_txtvec_121,about_me_field_txtvec_122,about_me_field_txtvec_123,about_me_field_txtvec_124,about_me_field_txtvec_125,about_me_field_txtvec_126,about_me_field_txtvec_127,about_me_field_txtvec_128,about_me_field_txtvec_129,about_me_field_txtvec_130,about_me_field_txtvec_131,about_me_field_txtvec_132,about_me_field_txtvec_133,about_me_field_txtvec_134,about_me_field_txtvec_135,about_me_field_txtvec_136,about_me_field_txtvec_137,about_me_field_txtvec_138,about_me_field_txtvec

# Prepare data for DataRobot and Upload

#### Drop irrelevant columns, check size of data

In [37]:
cols_to_drop = ['missed_pmts', 'nonfraud_default', 'sift_labeled_bad', 'sift_science_score', 
                'made_pmts', 'fraud_flag', 'reserve_fee_pct','country_id']
final_df = df.drop(cols_to_drop, axis=1)

In [38]:
#Combine the ids to obtain a unique user id
final_df['unique_id'] = final_df.borrower_id.astype(str) +'__'+ final_df.id.astype(str)
final_df.drop(['borrower_id', 'id'], axis = 1, inplace=True)

#### Save Data

In [ ]:
file_path = '/Users/hamelhusain/Google Drive/Team Shared Folder - Zidisha/Data/TextVec_Experiment.gz'
final_df.to_csv(file_path, compression = 'gzip', index=False)

#### Upload to DataRobot

In [ ]:
import datarobot as dr
proj = dr.Project.create(file_path, project_name='Text Vector With Text')

In [ ]:
proj.set_target(target = 'default_flag', 
                metric = 'AUC', 
                mode = dr.AUTOPILOT_MODE.QUICK,
                worker_count = 25)

#### Alternate version without original text fields

In [ ]:
final_df_notext = final_df.drop(['about_me_field', 'about_business_field', 'proposal_field'], axis =1)
proj2 = dr.Project.create(final_df_notext, project_name='Text Vector No Text')
proj2.set_target(target = 'default_flag', 
                metric = 'AUC', 
                mode = dr.AUTOPILOT_MODE.QUICK,
                worker_count = 25)